In [1]:
import math
import random

import pandas as pd
import numpy as np
from collections import defaultdict
import time
import pickle

from sklearn.model_selection import train_test_split
import joblib

import matplotlib.pyplot as plt
import seaborn as sns
import copy

from itertools import combinations
from datetime import datetime
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from Model import Retail_Recommendation

## Reading datasets
df = pd.read_parquet('../Data/data_with_features.parquet')
customer = pd.read_pickle('../Data/customer_history.pkl')
items = pd.read_pickle('../Data/item_summary.pkl')
baskets = pd.read_pickle('../Data/baskets.pkl')
itemsets = joblib.load('../Models/itemsets.joblib')
rules = pd.read_pickle('../Data/rules.pkl')
vectorizer = joblib.load('../Models/vectorizer.joblib')

# coefficients to train
params = {'alpha': 1.0, 'beta': 1.0, 'delta': 1.0, 'eta': 1.0, 'gamma': 0.1, 'epsilon': 2.0, 'd_effect': 0.5}

# Calculating parameters
Today = pd.Timestamp(df['Purchase Date'].max()) + pd.DateOffset(days=1)
Time_period = (df['Purchase Date'].max() - df['Purchase Date'].min()).days
d_effect = 0.5 # decay factor
EPS = 1e-9

Discount_dict = {} #{item: math.pow(np.random.default_rng().uniform(),(np.random.randint(1, 5))) for item in items.sample(100)['StockCode'].unique()}

Baskets = baskets[ baskets['Num products']>= 5].drop(columns=['Purchase Time', 'InvoiceDate', 'Quantity', 'Amount', 'Total amount'])
# we assume all purchases will be made today -> recency bias will not be trained well.
Baskets['X'] = Baskets['StockCode'].apply(lambda x: x[:-1])
Baskets['Y'] = Baskets['StockCode'].apply(lambda x: x[-1])


Baskets['Total Cost'] = Baskets['Price'].apply(np.sum)
# randomly scale up.down total cost to simulate budget
# ±20% variation (uniform)
variation = np.random.uniform(0.9, 1.25, size=len(Baskets))
Baskets['pseudo_budget'] = Baskets['Total Cost'] * variation

In [2]:
Baskets.sample(2)

,Invoice,Customer ID,Purchase Date,StockCode,Description,Price,Num products,X,Y,Total Cost,pseudo_budget
27415,569148,16613,2011-09-30,"[84946, 22960, 22867, 85123A, 21790, 21891, 21...","[antique|silver|t-light|glass, jam|making|set|...","[1.25, 4.25, 2.1, 2.95, 0.85, 1.45, 3.75, 1.25...",13,"[84946, 22960, 22867, 85123A, 21790, 21891, 21...",22086,31.55,38.878208
23492,557875,13300,2011-06-23,"[79321, 23209, 23203, 23202, 22720, 22630, 226...","[chilli|lights, lunch|bag|doiley|pattern, jumb...","[5.75, 1.65, 2.08, 2.08, 4.95, 1.95, 1.95, 1.6...",12,"[79321, 23209, 23203, 23202, 22720, 22630, 226...",22666,30.46,28.994124


In [3]:
## Random sampling -> train test split
train_baskets, test_baskets = train_test_split(Baskets, shuffle=True, test_size=0.3, random_state=42)
test_baskets, validation_baskets = train_test_split(test_baskets, shuffle=True, test_size=0.5, random_state=42)

## Time based splitting
df_dates = Baskets['Purchase Date']
cut1 = df_dates.quantile(0.7)
cut2 = df_dates.quantile(0.85)
train = Baskets[df_dates < cut1]
validation = Baskets[(df_dates >= cut1) & (df_dates < cut2)]
test = Baskets[df_dates >= cut2]

#### Part-7: Grid Search CV

Now we will train model for first few steps for various parameters and choose the best parameters for our model.

In [4]:
Grid_params = {
    # No need to include weights as they are trained already -> Just choose b/w 0 and non-zero
    ## -> Works like feature selection
    'alpha': [0.0,0.5],
    'beta': [0.0,0.5],
    'delta': [0.0,0.5],
    'gamma': [0.0,0.5],
    'eta': [0.0,0.5],
    'epsilon': [0.0,0.5],
    # model gradient weight
    'lr' : [0.01, 0.05, 0.1],
    
    'reg_lambda': [0.0, 0.05, 0.1],
    'lr_decay': [0.0, 0.05, 0.1],
    'clip_value': [0.0, 0.05, 0.1],
}

In [5]:
# previously: temp_model = copy.deepcopy(model)
# Instead, build a fresh model with the same constructor args:
import gc # garbage collector

def make_model_from_template(template_model, best_weights):
    gc.collect()
    
    new_model = Retail_Recommendation(
        items_data = template_model.item_data,
        customer_data = template_model.customer_data,
        rules = template_model.rules,
        vectorizer = getattr(template_model, 'vectorizer', None),
        initial_weights = copy.deepcopy(best_weights), 
        d_effect = template_model.d_effect,
        current_date = template_model.current_date,
        Half_life = template_model.half_life,
        Time_period = template_model.Time_period,
        filter_items = False,
        # Remove description feature for removing overfitting
        include_description = False, # template_model.include_description,
        iteration_bar = False
    )
    
    return new_model

1. Defining objective function

In [6]:
import optuna
import copy
import traceback

def objective(trial, model_template, best_weights, train_Data, val_data, top_n=100, n_iter=15, error=1e-6, batch_size=128, patience=5):
    """
    Objective function for Optuna hyperparameter search.
    Includes detailed logging and safe handling of crashes.
    """
    gc.collect()
    
    # Suggest hyperparameters
    lr = trial.suggest_float("lr", 0.05, 0.2, log=True)
    reg_lambda = trial.suggest_float("reg_lambda", 0.0, 1.0)
    lr_decay = trial.suggest_float("lr_decay", 0.0, 0.4)
    clip_value = trial.suggest_float("clip_value", 0.5, 5.0)
    
    # Deep copy the model so each trial is independent
    temp_model = make_model_from_template(model_template, best_weights)
    
    try:
        # Train
        print(f"[Trial {trial.number}] Starting training with lr={lr:.5f}, reg_lambda={reg_lambda:.5f}, lr_decay={lr_decay:.5f}, clip_value={clip_value:.2f}")
        weights, history = temp_model.train_model(
            Carts=train_Data, lr=lr, n_iter=n_iter, error=error, batch_size=batch_size, top_n=top_n,
            reg_lambda=reg_lambda, Learning_rate_decay=lr_decay, clip_value=clip_value,
            early_stopping=False, patience=patience, verbose=False
        )
        
        # Evaluate
        metrics = temp_model.Evaluate_model(val_data, top_n=top_n, batch_size=None)
        Hit_Rate = metrics.get("HitRate", 0)
        NDCG = metrics.get("NDCG@N", 0)
        score = 0.7*Hit_Rate + 0.3*NDCG if NDCG else Hit_Rate
        
        # Store metrics for debugging + score + weights
        trial.set_user_attr("metrics", metrics)
        trial.set_user_attr("params", weights)
        trial.set_user_attr("score", score)
        
        del temp_model
        gc.collect()
        return score
    
    except Exception as e:
        print(f"[Trial {trial.number}] Failed with exception:")
        traceback.print_exc()
        trial.set_user_attr("error", str(e))
        
        del temp_model
        gc.collect()
        return 0.0

2. Run the optuna study

In [7]:
def run_optuna_search(model, best_weights, train_data, val_data, n_trials=5):
    study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())
    
    study.optimize(
        lambda trial: objective(trial, model, best_weights, train_data, val_data), 
        n_trials=n_trials, show_progress_bar=True
    )
    
    print("\n Best trail:")
    Optimal_run = study.best_trial
    print("  Hit Rate: ", Optimal_run.value)
    print("  Best Parameters: ",Optimal_run.params)
    
    return study

3. Running trail: <br>
20 iteration for 20 trail with 2000 baskets is enough.

> Randomly sampling baskets for each trail is bad practice which leads to inconsistent results.

In [8]:
with open("../Models/best_parameters.pkl", "rb") as f:
    best_weights = pickle.load(f)

grid_model = Retail_Recommendation(
    items_data = items,
    customer_data = customer,
    rules = rules,
    initial_weights = best_weights,
    current_date = Today,
    Time_period = 100,
    filter_items = False,
    include_description = True,
    iteration_bar = False
)

study = run_optuna_search(
    model = grid_model, best_weights=best_weights, train_data=train.sample(train.shape[0]//4), val_data=validation, n_trials=20
)

[I 2025-10-21 10:38:29,599] A new study created in memory with name: no-name-acbf02ef-c55f-45a3-b81b-dbd627f86446


  0%|          | 0/20 [00:00<?, ?it/s]

[Trial 0] Starting training with lr=0.17710, reg_lambda=0.71671, lr_decay=0.07458, clip_value=3.29
[I 2025-10-21 10:39:01,780] Trial 0 finished with value: 0.3184748413782441 and parameters: {'lr': 0.17710428143382625, 'reg_lambda': 0.716707880544434, 'lr_decay': 0.07458338193651541, 'clip_value': 3.293837877709642}. Best is trial 0 with value: 0.3184748413782441.
[Trial 1] Starting training with lr=0.18871, reg_lambda=0.97034, lr_decay=0.25490, clip_value=3.17
[I 2025-10-21 10:39:33,348] Trial 1 finished with value: 0.2844307952627347 and parameters: {'lr': 0.18870850183409726, 'reg_lambda': 0.9703364277894366, 'lr_decay': 0.2548996428772405, 'clip_value': 3.1691441464342773}. Best is trial 0 with value: 0.3184748413782441.
[Trial 2] Starting training with lr=0.17787, reg_lambda=0.30966, lr_decay=0.11474, clip_value=1.24
[I 2025-10-21 10:40:05,226] Trial 2 finished with value: 0.30960384386108714 and parameters: {'lr': 0.1778711228062552, 'reg_lambda': 0.3096592151891341, 'lr_decay': 

4. Access Results

In [9]:
best_trial = study.best_trial
best_params = best_trial.params
best_hit_rate = best_trial.value
best_metrics = best_trial.user_attrs.get("metrics", {})

print("Best Hit Rate:", best_hit_rate)
print("Best Parameters:", best_params)
print("Metrics:", best_metrics)

print("Best Hit Rate: ",best_hit_rate)

Best Hit Rate: 0.35000647643063665
Best Parameters: {'lr': 0.10679514829809283, 'reg_lambda': 0.12940326697938503, 'lr_decay': 0.018390424287397567, 'clip_value': 4.967950851842016}
Metrics: {'Precision@N': np.float64(0.004357859531772576), 'Recall@N': np.float64(0.43578595317725755), 'F1@N': np.float64(0.00862942472995139), 'HitRate': 0.43578595317725755, 'MRR@N': np.float64(0.07579494359667348), 'NDCG@N': np.float64(0.14985436402185462), 'MAP@N': np.float64(0.07579494359667348), 'Correct recommendations': 1303, 'Fraction correct as ratio': '1303/2990'}
Best Hit Rate:  0.35000647643063665


In [10]:
optuna.visualization.plot_optimization_history(study).show()

In [11]:
optuna.visualization.plot_param_importances(study).show()

Saving all these results:

In [12]:
import pickle

with open("../experiments/optuna_study.pkl", "wb") as f:
    pickle.dump(study, f)

In [13]:
# Convert trials to DataFrame
df_trials = study.trials_dataframe()
df_trials.to_csv("../experiments/optuna_trials.csv", index=False)
print("All trial results saved to optuna_trials.csv")

All trial results saved to optuna_trials.csv


Now, Evaluation of Best model:
1. Weights from training model
2. model parameters from best trail

In [14]:
# === 1. Load best model weights ===
best_weights = best_trial.user_attrs.get("params", {})

# === 2. Load optuna study and best hyperparameters ===
with open("../experiments/optuna_study.pkl", "rb") as f:
    loaded_study = pickle.load(f)

best_trial = loaded_study.best_trial
best_params = best_trial.params

from Model import Retail_Recommendation

# === 3. Build new model with best weights ===
best_model = Retail_Recommendation(
    items_data = items,
    customer_data = customer,
    rules = rules,
    initial_weights = best_weights,
    d_effect= best_weights.get("d_effect", 0.5),
    current_date = Today,
    Time_period = 100,
    filter_items = False,
    include_description = True,
    iteration_bar = False
)

# === 4. Retrain using best hyperparameters from Optuna ===
final_weights, history = best_model.train_model(
    Carts=train,
    lr=best_params.get('lr', 0.05),
    reg_lambda=best_params.get('reg_lambda', 0.0),
    Learning_rate_decay=best_params.get('lr_decay', 0.0),
    clip_value=best_params.get('clip_value', None),
    n_iter=30,   # or increase for final run
    error=1e-6,
    batch_size=256,
    early_stopping=True,
    patience=3,
    verbose=True
)

Epoch 1/50: 100%|██████████| 55/55 [00:20<00:00,  2.62it/s]


Epoch 1/50 | Loss: 5.946566 
self.weights: {'alpha': np.float32(0.5859), 'beta': np.float32(4.7805), 'delta': np.float32(0.6985), 'eta': np.float32(0.5859), 'gamma': np.float32(0.0586), 'epsilon': np.float32(7.4163), 'd_effect': 1}
Correct predictions:  12571/13861 


Epoch 2/50: 100%|██████████| 55/55 [00:19<00:00,  2.81it/s]


Epoch 2/50 | Loss: 5.763265 
self.weights: {'alpha': np.float32(0.5822), 'beta': np.float32(4.282), 'delta': np.float32(0.6911), 'eta': np.float32(0.5822), 'gamma': np.float32(0.0582), 'epsilon': np.float32(7.5313), 'd_effect': 1}
Correct predictions:  12584/13861 


Epoch 3/50: 100%|██████████| 55/55 [00:20<00:00,  2.70it/s]


...

Epoch 48/50 | Loss: 4.495498 
self.weights: {'alpha': np.float32(0.2669), 'beta': np.float32(-0.3324), 'delta': np.float32(0.657), 'eta': np.float32(0.2669), 'gamma': np.float32(0.0267), 'epsilon': np.float32(12.3359), 'd_effect': 1}
Correct predictions:  12681/13861 


Epoch 49/50: 100%|██████████| 55/55 [00:19<00:00,  2.87it/s]


Epoch 49/50 | Loss: 4.495441 
self.weights: {'alpha': np.float32(0.263), 'beta': np.float32(-0.333), 'delta': np.float32(0.6582), 'eta': np.float32(0.263), 'gamma': np.float32(0.0263), 'epsilon': np.float32(12.3366), 'd_effect': 1}
Correct predictions:  12681/13861 


Epoch 50/50: 100%|██████████| 55/55 [00:20<00:00,  2.64it/s]


Epoch 50/50 | Loss: 4.495391 
self.weights: {'alpha': np.float32(0.2592), 'beta': np.float32(-0.3335), 'delta': np.float32(0.6593), 'eta': np.float32(0.2592), 'gamma': np.float32(0.0259), 'epsilon': np.float32(12.3372), 'd_effect': 1}
Correct predictions:  12681/13861 


In [15]:
# === 4. Evaluate best model on test data ===
metrics = best_model.Evaluate_model(test, top_n=100)
print("Metrics:", metrics)
print("Hit Rate:", metrics.get("HitRate", 0))

# === 5. Display full evalutaion metrics ===
print("\nFull evaluation metrics:")
for metric, value in metrics.items():
    print(f"{metric}: {value}")

Metrics: {'Precision@N': np.float64(0.0096875), 'Recall@N': np.float64(0.96875), 'F1@N': np.float64(0.019183168126899325), 'HitRate': 0.04125083166999335, 'MRR@N': np.float64(0.2299286862308245), 'NDCG@N': np.float64(0.38035014977718007), 'MAP@N': np.float64(0.2299286862308245), 'Correct recommendations': 124, 'Fraction correct as ratio': '124/3006'}
Hit Rate: 0.04125083166999335

Full evaluation metrics:
Precision@N: 0.0096875
Recall@N: 0.96875
F1@N: 0.019183168126899325
HitRate: 0.04125083166999335
MRR@N: 0.2299286862308245
NDCG@N: 0.38035014977718007
MAP@N: 0.2299286862308245
Correct recommendations: 124
Fraction correct as ratio: 124/3006


In [16]:
# === 6. Logging summary for record ===
summary = {
    "Best hit rate": metrics.get("HitRate", 0),
    "Best Feature weights": final_weights,
    "Best hyperparameters": best_params,
    "Metrics": metrics
}
print(summary)


with open("../experiments/Best_result.pkl", "wb") as f:
    pickle.dump(summary, f)

print("\n✅ Evaluation summary saved to ../experiments/Best_result.pkl")

{'Best hit rate': 0.04125083166999335, 'Best Feature weights': {'alpha': np.float32(0.25923294), 'beta': np.float32(-0.33347318), 'delta': np.float32(0.6593087), 'eta': np.float32(0.25923294), 'gamma': np.float32(0.025923286), 'epsilon': np.float32(12.337225), 'd_effect': 1}, 'Best hyperparameters': {'lr': 0.10679514829809283, 'reg_lambda': 0.12940326697938503, 'lr_decay': 0.018390424287397567, 'clip_value': 4.967950851842016}, 'Metrics': {'Precision@N': np.float64(0.0096875), 'Recall@N': np.float64(0.96875), 'F1@N': np.float64(0.019183168126899325), 'HitRate': 0.04125083166999335, 'MRR@N': np.float64(0.2299286862308245), 'NDCG@N': np.float64(0.38035014977718007), 'MAP@N': np.float64(0.2299286862308245), 'Correct recommendations': 124, 'Fraction correct as ratio': '124/3006'}}

✅ Evaluation summary saved to ../experiments/Best_result.pkl


In [19]:
import pickle
with open("../experiments/Best_result.pkl", "rb") as f:
    summary = pickle.load(f)
summary

{'Best hit rate': 0.04125083166999335,
 'Best Feature weights': {'alpha': np.float32(0.25923294),
  'beta': np.float32(-0.33347318),
  'delta': np.float32(0.6593087),
  'eta': np.float32(0.25923294),
  'gamma': np.float32(0.025923286),
  'epsilon': np.float32(12.337225),
  'd_effect': 1},
 'Best hyperparameters': {'lr': 0.10679514829809283,
  'reg_lambda': 0.12940326697938503,
  'lr_decay': 0.018390424287397567,
  'clip_value': 4.967950851842016},
 'Metrics': {'Precision@N': np.float64(0.0096875),
  'Recall@N': np.float64(0.96875),
  'F1@N': np.float64(0.019183168126899325),
  'HitRate': 0.04125083166999335,
  'MRR@N': np.float64(0.2299286862308245),
  'NDCG@N': np.float64(0.38035014977718007),
  'MAP@N': np.float64(0.2299286862308245),
  'Correct recommendations': 124,
  'Fraction correct as ratio': '124/3006'}}

Now, we will always use these best parameters for our model.

In [20]:
with open("../Models/Final_model.pkl", "wb") as f:
    pickle.dump(best_model, f)